# Aurora Forecasting - Part 01: Feature Backfill

🗒️ This notebook is divided into the following sections:
Initialize Hopsworks connection.

Fetch historical Solar Wind & Kp index data using spacepy (OMNI dataset).

Fetch historical Cloud Cover for Stockholm, Luleå, and Kiruna using Open-Meteo.

Create and Insert data into Feature Groups in the Hopsworks Feature Store.

# Imports

In [13]:
!pip install spacepy cdasws


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [14]:
import pandas as pd
import datetime
import hopsworks
from config import HopsworksSettings
import util
import spacepy.omni as omni
import spacepy.time as spt
import spacepy.toolbox as tb
from cdasws import CdasWs
from cdasws.timeinterval import TimeInterval

# Setup settings
settings = HopsworksSettings()

HopsworksSettings initialized!


# Step 1: Historical Solar Wind & Kp Data

We use the OMNI dataset via spacepy to get high-resolution historical satellite data. This includes the magnetic field components and the proton parameters needed to predict the Kp index.

https://helio.data.nasa.gov/dataset/OMNI_PT1H 

In [15]:
cdas = CdasWs()

# Get the OMNI dataset
#dataset = 'OMNI_HRO_1MIN'  # 1 minute resolution
dataset = 'OMNI2_H0_MRG1HR'  # 1 hour resolution
var_names = cdas.get_variable_names(dataset)

print('Variable names:', var_names)

# Time interval
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=3650)  # 6 years

# cdas.get_data expects a cdasws TimeInterval (or ISO start/end strings).
# Build a TimeInterval from the start and end datetimes to avoid "invalid time0 type".
time_interval = TimeInterval(start_date.strftime('%Y-%m-%dT%H:%M:%S'),
                             end_date.strftime('%Y-%m-%dT%H:%M:%S'))

# Fetch data
status, data = cdas.get_data(dataset, var_names, time_interval)

# Keep this (suggested in the official site)
if 'spacepy' in str(type(data)):
    #  see https://spacepy.github.io/datamodel.html
    print(var_names[0], '=', data[var_names[0]])
    print(data[var_names[0]].attrs)
    for var in var_names:
        print(var, '=', data[var])
        print(data[var].attrs)
        print("\n")
else:
    #  see https://github.com/MAVENSDC/cdflib
    print(var_names[0], '=', data.data_vars[var_names[0]].values)
    print(data.data_vars[var_names[0]].attrs)


Variable names: ['Rot1800', 'IMF1800', 'PLS1800', 'IMF_PTS1800', 'PLS_PTS1800', 'ABS_B1800', 'F1800', 'THETA_AV1800', 'PHI_AV1800', 'BX_GSE1800', 'BY_GSE1800', 'BZ_GSE1800', 'BY_GSM1800', 'BZ_GSM1800', 'SIGMA-ABS_B1800', 'SIGMA-B1800', 'SIGMA-Bx1800', 'SIGMA-By1800', 'SIGMA-Bz1800', 'T1800', 'N1800', 'V1800', 'PHI-V1800', 'THETA-V1800', 'Ratio1800', 'Pressure1800', 'SIGMA-T1800', 'SIGMA-N1800', 'SIGMA-V1800', 'SIGMA-PHI-V1800', 'SIGMA-THETA-V1800', 'SIGMA-ratio1800', 'E1800', 'Beta1800', 'Mach_num1800', 'Mgs_mach_num1800', 'PR-FLX_11800', 'PR-FLX_21800', 'PR-FLX_41800', 'PR-FLX_101800', 'PR-FLX_301800', 'PR-FLX_601800', 'MFLX1800', 'R1800', 'F10_INDEX1800', 'KP1800', 'DST1800', 'AE1800', 'AP_INDEX1800', 'AL_INDEX1800', 'AU_INDEX1800', 'PC_N_INDEX1800', 'Solar_Lyman_alpha1800', 'Proton_QI1800']


Rot1800 = [2489 2489 2489 ... 9999 9999 9999]
{'FIELDNAM': 'Bartels Rotation Number', 'VALIDMIN': 1700, 'VALIDMAX': 9998, 'SCALEMIN': 1700, 'SCALEMAX': 9998, 'FORMAT': 'I4', 'FILLVAL': 9999, 'VAR_TYPE': 'data', 'CATDESC': 'Bartels Rotation Number (half-hour midpoint)', 'DISPLAY_TYPE': 'time_series', 'DEPEND_0': 'Epoch_1800', 'LABLAXIS': 'Bartel #', 'BIN_LOCATION': '0.5', 'DIM_SIZES': 0}
Rot1800 = [2489 2489 2489 ... 9999 9999 9999]
{'FIELDNAM': 'Bartels Rotation Number', 'VALIDMIN': 1700, 'VALIDMAX': 9998, 'SCALEMIN': 1700, 'SCALEMAX': 9998, 'FORMAT': 'I4', 'FILLVAL': 9999, 'VAR_TYPE': 'data', 'CATDESC': 'Bartels Rotation Number (half-hour midpoint)', 'DISPLAY_TYPE': 'time_series', 'DEPEND_0': 'Epoch_1800', 'LABLAXIS': 'Bartel #', 'BIN_LOCATION': '0.5', 'DIM_SIZES': 0}


IMF1800 = [51 51 51 ... 99 99 99]
{'FIELDNAM': 'ID for IMF spacecraft', 'VALIDMIN': 1, 'VALIDMAX': 98, 'SCALEMIN': 1, 'SCALEMAX': 98, 'FORMAT': 'I2', 'FILLVAL': 99, 'VAR_TYPE': 'data', 'CATDESC': 'OMNI ID code for the 

In [16]:
# Extract needed information into a pd frame
solar_wind_df = pd.DataFrame({
    'date_and_time': data['Epoch'],
    'by_gsm': data['BY_GSM1800'],
    'bz_gsm': data['BZ_GSM1800'],
    'density': data['N1800'],
    'speed': data['V1800'],
    'kp_index': data['KP1800']
})

print(f"Extracted {len(solar_wind_df)} records")
solar_wind_df.tail(200)

Extracted 87228 records


,date_and_time,by_gsm,bz_gsm,density,speed,kp_index
87028,2025-12-18 05:00:00,999.900024,999.900024,2.200000,689.0,27
87029,2025-12-18 06:00:00,999.900024,999.900024,2.200000,688.0,37
87030,2025-12-18 07:00:00,999.900024,999.900024,2.400000,676.0,37
87031,2025-12-18 08:00:00,999.900024,999.900024,2.300000,651.0,37
87032,2025-12-18 09:00:00,999.900024,999.900024,2.300000,638.0,33
...,...,...,...,...,...,...
87223,2025-12-26 08:00:00,999.900024,999.900024,999.900024,9999.0,99
87224,2025-12-26 09:00:00,999.900024,999.900024,999.900024,9999.0,99
87225,2025-12-26 10:00:00,999.900024,999.900024,999.900024,9999.0,99
87226,2025-12-26 11:00:00,999.900024,999.900024,999.900024,9999.0,99


In [17]:
# Data Cleaning: OMNI uses 99.9 or 999.9 as fill values for missing data
solar_wind_df = solar_wind_df[(solar_wind_df['kp_index'] < 99) & 
                        (solar_wind_df['by_gsm'] < 999.9) &
                        (solar_wind_df['bz_gsm'] < 999.9) &
                        (solar_wind_df['density'] < 999.9) &
                        (solar_wind_df['speed'] < 9999.9)
                        ]
solar_wind_df = solar_wind_df.dropna()

# Divide by 10 the kp index to get the real value
solar_wind_df['kp_index'] = solar_wind_df['kp_index'] / 10.0
solar_wind_df['kp_index'] = solar_wind_df['kp_index'].astype('float32')

# Transform data column in string for Hopswork feature perchè se no hopsworks rompe i coglioni che non supporta timestamp come primary key
#solar_wind_df['time'] = solar_wind_df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

print(solar_wind_df.dtypes)
solar_wind_df


date_and_time    datetime64[ns]
by_gsm                  float32
bz_gsm                  float32
density                 float32
speed                   float32
kp_index                float32
dtype: object


,date_and_time,by_gsm,bz_gsm,density,speed,kp_index
0,2016-01-14 01:00:00,-1.9,1.7,2.7,510.0,1.3
1,2016-01-14 02:00:00,-2.2,-2.8,2.7,532.0,1.3
2,2016-01-14 03:00:00,-1.6,-1.5,2.6,514.0,2.0
3,2016-01-14 04:00:00,-1.0,1.5,2.8,525.0,2.0
4,2016-01-14 05:00:00,-1.2,0.3,2.4,490.0,2.0
...,...,...,...,...,...,...
86828,2025-12-09 21:00:00,-0.0,-2.6,4.0,351.0,1.3
86829,2025-12-09 22:00:00,-1.4,-2.2,3.8,347.0,1.3
86830,2025-12-09 23:00:00,-2.0,-2.1,4.0,345.0,1.3
86831,2025-12-10 00:00:00,-0.9,-2.2,2.9,335.0,2.0


In [18]:
# Calculate and add the dynamic pressure to the dataframe
solar_wind_df = util.calculate_dynamic_pressure(solar_wind_df)
solar_wind_df

,date_and_time,by_gsm,bz_gsm,density,speed,kp_index,dynamic_pressure
0,2016-01-14 01:00:00,-1.9,1.7,2.7,510.0,1.3,702270.0000
1,2016-01-14 02:00:00,-2.2,-2.8,2.7,532.0,1.3,764164.8125
2,2016-01-14 03:00:00,-1.6,-1.5,2.6,514.0,2.0,686909.5625
3,2016-01-14 04:00:00,-1.0,1.5,2.8,525.0,2.0,771750.0000
4,2016-01-14 05:00:00,-1.2,0.3,2.4,490.0,2.0,576240.0000
...,...,...,...,...,...,...,...
86828,2025-12-09 21:00:00,-0.0,-2.6,4.0,351.0,1.3,492804.0000
86829,2025-12-09 22:00:00,-1.4,-2.2,3.8,347.0,1.3,457554.1875
86830,2025-12-09 23:00:00,-2.0,-2.1,4.0,345.0,1.3,476100.0000
86831,2025-12-10 00:00:00,-0.9,-2.2,2.9,335.0,2.0,325452.5000


In [19]:
# 3H aggregation for Kp physical correctness. 
# Kp index is repeated for each 3H window, but it's not correct for training otherwise.

# Use the reusable function from util.py
solar_wind_aggregated_df = util.aggregate_solar_wind_3h(
    df=solar_wind_df,
    time_col='date_and_time',
    feature_cols=['by_gsm', 'bz_gsm', 'density', 'speed', 'dynamic_pressure'],
    target_col='kp_index',
    min_samples=2
)

print(f"After 3H aggregation: {len(solar_wind_aggregated_df)} records")
print("\nNew columns:")
print(solar_wind_aggregated_df.columns.tolist())
print("\nData types:")
print(solar_wind_aggregated_df.dtypes)
print("\nSample data showing 3H windows:")
solar_wind_aggregated_df

2026-01-11 00:49:49,161 WARNING: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.

2026-01-11 00:49:49,170 WARNING: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.

2026-01-11 00:49:49,213 WARNING: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.

After 3H aggregation: 28653 records

New columns:
['window_start', 'window_end', 'by_gsm_mean', 'by_gsm_min', 'by_gsm_max', 'by_gsm_std', 'bz_gsm_mean', 'bz_gsm_min', 'bz_gsm_max', 'bz_gsm_std', 'density_mean', 'density_min', 'density_max', 'density_std', 'speed_mean', 'speed_min', 'speed_max', 'speed_std', 'dynamic_pressure_mean', 'dynamic_pressure_min', 'dynamic_pressure_max', 'dynamic_pressure_std', 'kp_index']

Data types:
window_start             datetime64[ns]
window_end               datetime64[ns]
by_gsm_mean                     float32
by_gsm_min                      float32
by_gsm_max   

,window_start,window_end,by_gsm_mean,by_gsm_min,by_gsm_max,by_gsm_std,bz_gsm_mean,bz_gsm_min,bz_gsm_max,bz_gsm_std,...,density_std,speed_mean,speed_min,speed_max,speed_std,dynamic_pressure_mean,dynamic_pressure_min,dynamic_pressure_max,dynamic_pressure_std,kp_index
0,2016-01-14 00:00:00,2016-01-14 03:00:00,-2.050000,-2.2,-1.9,0.212132,-0.550000,-2.8,1.7,3.181981,...,0.000000,521.000000,510.0,532.0,15.556349,733217.37500,702270.00000,764164.81250,43766.241639,1.3
1,2016-01-14 03:00:00,2016-01-14 06:00:00,-1.266667,-1.6,-1.0,0.305505,0.100000,-1.5,1.5,1.509967,...,0.200000,509.666656,490.0,525.0,17.897858,678299.81250,576240.00000,771750.00000,98038.948042,2.0
2,2016-01-14 06:00:00,2016-01-14 09:00:00,-3.066667,-4.0,-1.9,1.069268,-1.433333,-3.1,1.0,2.154840,...,0.115470,514.000000,501.0,535.0,18.357559,704562.81250,652602.56250,744185.00000,47021.333277,2.3
3,2016-01-14 09:00:00,2016-01-14 12:00:00,-0.900000,-3.2,1.3,2.251666,0.700000,-1.1,2.2,1.670329,...,0.100000,502.333344,493.0,507.0,8.082904,706203.81250,694032.31250,719737.18750,12906.429483,2.0
4,2016-01-14 12:00:00,2016-01-14 15:00:00,-1.500000,-2.4,-0.7,0.854400,-0.500000,-1.9,0.9,1.400000,...,0.251661,468.000000,454.0,480.0,13.114877,564508.93750,474066.78125,645120.00000,85949.336281,1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28648,2025-12-09 12:00:00,2025-12-09 15:00:00,1.700000,1.4,2.0,0.424264,-3.300000,-3.4,-3.2,0.141421,...,0.141421,345.000000,345.0,345.0,0.000000,166635.00000,154732.50000,178537.50000,16832.676926,1.0
28649,2025-12-09 15:00:00,2025-12-09 18:00:00,2.000000,1.2,3.3,1.135782,-3.500000,-4.0,-2.9,0.556776,...,0.251661,365.333344,353.0,372.0,10.692677,422680.62500,398748.81250,467979.40625,39250.859362,1.3
28650,2025-12-09 18:00:00,2025-12-09 21:00:00,-1.700000,-3.2,-0.4,1.410674,-2.133333,-2.6,-1.8,0.416333,...,0.450925,368.000000,360.0,376.0,8.000000,417196.78125,336960.00000,494816.00000,78960.547053,1.3
28651,2025-12-09 21:00:00,2025-12-10 00:00:00,-1.133333,-2.0,-0.0,1.026320,-2.300000,-2.6,-2.1,0.264575,...,0.115470,347.666656,345.0,351.0,3.055050,475486.09375,457554.18750,492804.00000,17632.924028,1.3


In [20]:
# Sort based on the date and time to be sure and reset the index
solar_wind_df = solar_wind_df.sort_values("date_and_time").reset_index(drop=True)

print(f"Extracted {len(solar_wind_df)} records")
# Drop the column of the KP index, because it is not useful for the inference
solar_wind_df = solar_wind_df.drop(columns=['kp_index'])

Extracted 85916 records


# Step 3: Create Feature Groups and Insert Data

Now we register these datasets in the Hopsworks Feature Store.

We transformed these values here because they are model-independent, not model-dependent. So according to the O'Really book it is legal.

In [21]:
# Login to Hopsworks
project = hopsworks.login(
    project=settings.HOPSWORKS_PROJECT,
    api_key_value=settings.HOPSWORKS_API_KEY.get_secret_value()
)
fs = project.get_feature_store()

2026-01-11 00:49:49,277 INFO: Closing external client and cleaning up certificates.
Connection closed.
2026-01-11 00:49:49,280 INFO: Initializing external client
2026-01-11 00:49:49,281 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-11 00:49:49,905 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2026-01-11 00:49:50,611 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1299605


In [22]:
print("Before casting the aggregated data:\n", solar_wind_aggregated_df)
# Clean and cast to correct types for Feature Store compatibility
# Convert numeric columns to float32 (Feature Store expects 'float' not 'double')
df = solar_wind_aggregated_df.copy()

for col in df.columns:
    if col not in ["window_start", "window_end"]:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('float32')

solar_wind_aggregated_df = df
# check data types of each column
print("After casting:\n", solar_wind_aggregated_df.dtypes)
solar_wind_aggregated_df

Before casting the aggregated data:
 

             window_start          window_end  by_gsm_mean  by_gsm_min  \
0     2016-01-14 00:00:00 2016-01-14 03:00:00    -2.050000        -2.2   
1     2016-01-14 03:00:00 2016-01-14 06:00:00    -1.266667        -1.6   
2     2016-01-14 06:00:00 2016-01-14 09:00:00    -3.066667        -4.0   
3     2016-01-14 09:00:00 2016-01-14 12:00:00    -0.900000        -3.2   
4     2016-01-14 12:00:00 2016-01-14 15:00:00    -1.500000        -2.4   
...                   ...                 ...          ...         ...   
28648 2025-12-09 12:00:00 2025-12-09 15:00:00     1.700000         1.4   
28649 2025-12-09 15:00:00 2025-12-09 18:00:00     2.000000         1.2   
28650 2025-12-09 18:00:00 2025-12-09 21:00:00    -1.700000        -3.2   
28651 2025-12-09 21:00:00 2025-12-10 00:00:00    -1.133333        -2.0   
28652 2025-12-10 00:00:00 2025-12-10 03:00:00    -1.450000        -2.0   

       by_gsm_max  by_gsm_std  bz_gsm_mean  bz_gsm_min  bz_gsm_max  \
0            -1.9    0.212132    -0.55000

,window_start,window_end,by_gsm_mean,by_gsm_min,by_gsm_max,by_gsm_std,bz_gsm_mean,bz_gsm_min,bz_gsm_max,bz_gsm_std,...,density_std,speed_mean,speed_min,speed_max,speed_std,dynamic_pressure_mean,dynamic_pressure_min,dynamic_pressure_max,dynamic_pressure_std,kp_index
0,2016-01-14 00:00:00,2016-01-14 03:00:00,-2.050000,-2.2,-1.9,0.212132,-0.550000,-2.8,1.7,3.181981,...,0.000000,521.000000,510.0,532.0,15.556349,733217.37500,702270.00000,764164.81250,43766.242188,1.3
1,2016-01-14 03:00:00,2016-01-14 06:00:00,-1.266667,-1.6,-1.0,0.305505,0.100000,-1.5,1.5,1.509967,...,0.200000,509.666656,490.0,525.0,17.897858,678299.81250,576240.00000,771750.00000,98038.945312,2.0
2,2016-01-14 06:00:00,2016-01-14 09:00:00,-3.066667,-4.0,-1.9,1.069268,-1.433333,-3.1,1.0,2.154840,...,0.115470,514.000000,501.0,535.0,18.357559,704562.81250,652602.56250,744185.00000,47021.332031,2.3
3,2016-01-14 09:00:00,2016-01-14 12:00:00,-0.900000,-3.2,1.3,2.251666,0.700000,-1.1,2.2,1.670329,...,0.100000,502.333344,493.0,507.0,8.082904,706203.81250,694032.31250,719737.18750,12906.429688,2.0
4,2016-01-14 12:00:00,2016-01-14 15:00:00,-1.500000,-2.4,-0.7,0.854400,-0.500000,-1.9,0.9,1.400000,...,0.251661,468.000000,454.0,480.0,13.114877,564508.93750,474066.78125,645120.00000,85949.335938,1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28648,2025-12-09 12:00:00,2025-12-09 15:00:00,1.700000,1.4,2.0,0.424264,-3.300000,-3.4,-3.2,0.141421,...,0.141421,345.000000,345.0,345.0,0.000000,166635.00000,154732.50000,178537.50000,16832.677734,1.0
28649,2025-12-09 15:00:00,2025-12-09 18:00:00,2.000000,1.2,3.3,1.135782,-3.500000,-4.0,-2.9,0.556776,...,0.251661,365.333344,353.0,372.0,10.692677,422680.62500,398748.81250,467979.40625,39250.859375,1.3
28650,2025-12-09 18:00:00,2025-12-09 21:00:00,-1.700000,-3.2,-0.4,1.410674,-2.133333,-2.6,-1.8,0.416333,...,0.450925,368.000000,360.0,376.0,8.000000,417196.78125,336960.00000,494816.00000,78960.546875,1.3
28651,2025-12-09 21:00:00,2025-12-10 00:00:00,-1.133333,-2.0,-0.0,1.026320,-2.300000,-2.6,-2.1,0.264575,...,0.115470,347.666656,345.0,351.0,3.055050,475486.09375,457554.18750,492804.00000,17632.923828,1.3


In [23]:
print("Before casting the real time data:\n", solar_wind_df)
# Clean and cast to correct types for Feature Store compatibility
# Convert numeric columns to float32 (Feature Store expects 'float' not 'double')
df = solar_wind_df.copy()

for col in df.columns:
    if col not in ["date_and_time"]:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('float32')

solar_wind_df = df
# check data types of each column
print("After casting:\n", solar_wind_df.dtypes)
solar_wind_df

Before casting the real time data:
             date_and_time  by_gsm  bz_gsm  density  speed  dynamic_pressure
0     2016-01-14 01:00:00    -1.9     1.7      2.7  510.0       702270.0000
1     2016-01-14 02:00:00    -2.2    -2.8      2.7  532.0       764164.8125
2     2016-01-14 03:00:00    -1.6    -1.5      2.6  514.0       686909.5625
3     2016-01-14 04:00:00    -1.0     1.5      2.8  525.0       771750.0000
4     2016-01-14 05:00:00    -1.2     0.3      2.4  490.0       576240.0000
...                   ...     ...     ...      ...    ...               ...
85911 2025-12-09 21:00:00    -0.0    -2.6      4.0  351.0       492804.0000
85912 2025-12-09 22:00:00    -1.4    -2.2      3.8  347.0       457554.1875
85913 2025-12-09 23:00:00    -2.0    -2.1      4.0  345.0       476100.0000
85914 2025-12-10 00:00:00    -0.9    -2.2      2.9  335.0       325452.5000
85915 2025-12-10 01:00:00    -2.0    -2.2      3.0  331.0       328683.0000

[85916 rows x 6 columns]
After casting:
 date_and_t

,date_and_time,by_gsm,bz_gsm,density,speed,dynamic_pressure
0,2016-01-14 01:00:00,-1.9,1.7,2.7,510.0,702270.0000
1,2016-01-14 02:00:00,-2.2,-2.8,2.7,532.0,764164.8125
2,2016-01-14 03:00:00,-1.6,-1.5,2.6,514.0,686909.5625
3,2016-01-14 04:00:00,-1.0,1.5,2.8,525.0,771750.0000
4,2016-01-14 05:00:00,-1.2,0.3,2.4,490.0,576240.0000
...,...,...,...,...,...,...
85911,2025-12-09 21:00:00,-0.0,-2.6,4.0,351.0,492804.0000
85912,2025-12-09 22:00:00,-1.4,-2.2,3.8,347.0,457554.1875
85913,2025-12-09 23:00:00,-2.0,-2.1,4.0,345.0,476100.0000
85914,2025-12-10 00:00:00,-0.9,-2.2,2.9,335.0,325452.5000


In [24]:
# 1. Create Solar Wind aggregated Feature Group
solar_wind_aggregated_fg = fs.get_or_create_feature_group(
    name="solar_wind_aggregated_fg",
    version=3,
    primary_key=['window_start', 'window_end'],
    event_time="window_start",
    description="Satellite measurements (Bz, speed, density) and Kp index labels aggregated over 3H windows to match Kp physical meaning",
    #online_enabled=True,
    statistics_config={"enabled": True, "histograms": True, "correlations": True}
)

# 2. Create Solar Wind characteristic for each day
solar_wind_fg = fs.get_or_create_feature_group(
    name="solar_wind_fg",
    version=8,
    primary_key=['date_and_time'],
    event_time="date_and_time",
    description="Create Solar Wind characteristic for each day",
    #online_enabled=True,
    statistics_config={"enabled": True, "histograms": True}
)

# Insert Data
solar_wind_fg.insert(solar_wind_df)
solar_wind_aggregated_fg.insert(solar_wind_aggregated_df)

print("Backfill Complete! Data is now in the Hopsworks Feature Store.")

Uploading Dataframe: 100.00% |██████████| Rows 85916/85916 | Elapsed Time: 00:25 | Remaining Time: 00:00


Launching job: solar_wind_fg_8_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1299605/jobs/named/solar_wind_fg_8_offline_fg_materialization/executions


Uploading Dataframe: 100.00% |██████████| Rows 28653/28653 | Elapsed Time: 00:02 | Remaining Time: 00:00


2026-01-11 00:50:41,471 WARNING: UserWarning: Materialization job is already running, aborting new execution.Please wait for the current execution to finish before triggering a new one.You can check the status of the current execution using `fg.materialization_job.get_state()`.or `fg.materialization_job.get_final_state()` or check it out in the Hopsworks UI.at https://c.app.hopsworks.ai:443/p/1299605/jobs/named/solar_wind_aggregated_fg_3_offline_fg_materialization.
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/mac64/Resources/jobs/solar_wind_aggregated_fg_3_offline_fg_materialization/config_1768088878530) to trigger the materialization job again.

Backfill Complete! Data is now in the Hopsworks Feature Store.
